# Welcome to the 311 Data science team
## Please follow the answers to this tutorial if you are unsure how to proceed. Remember, there are multiple methods to completing the same taks in data cleanning!

This notebook has been adapated from Adel Sader's original notebook.

In [15]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
import re
import matplotlib.pyplot as plt
import os


In [16]:
# Enter your base directory
BASE_DIR = '/Users/you_computer/Downloads/datafile_location/'

<font size=3 color='DarkBlue'>311data.csv has been downloaded from [this source](https://dev.socrata.com/foundry/data.lacity.org/rq3b-xjk8). Please download [this file]() to your computer to run this script.

In [17]:
# navigate to the file location and read your csv file
os.chdir(BASE_DIR)

df311 = pd.read_csv('311data.csv')


/Users/karinalopez/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [18]:
df311.shape

(1491773, 35)

In [19]:
df311.head(2)

,Unnamed: 0,location,zipcode,suffix,srnumber,updateddate,closeddate,cd,address,createdbyuserorganization,...,status,tbmrow,direction,anonymous,addressverified,actiontaken,streetname,ncname,apc,tbmpage
0,0,"{'latitude': '33.9589038232', 'needs_recoding'...",90003,ST,1-1831739761,2021-01-02T08:12:27.000,2021-01-02T07:39:24.000,8.0,"114 E 87TH ST, 90003",Self Service,...,Closed,3.0,E,N,Y,SR Created,87TH,Empowerment Congress Southeast,South Los Angeles APC,704.0
1,1,"{'latitude': '34.2838039863', 'needs_recoding'...",91342,AVE,1-1831739741,2021-01-01T13:38:06.000,2021-01-01T13:38:06.000,7.0,"11743 N BRUSSELS AVE, 91342",Self Service_SAN,...,Cancelled,7.0,N,N,Y,SR Created,BRUSSELS,Foothill Trails District,North Valley APC,482.0


In [20]:
# drop your unnamed column
df311 = df311.drop(columns = ['Unnamed: 0'])

In [21]:
df311.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1491773 entries, 0 to 1491772
Data columns (total 34 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   location                   1490433 non-null  object 
 1   zipcode                    1490127 non-null  object 
 2   suffix                     1379792 non-null  object 
 3   srnumber                   1491773 non-null  object 
 4   updateddate                1491773 non-null  object 
 5   closeddate                 1482583 non-null  object 
 6   cd                         1490425 non-null  float64
 7   address                    1491707 non-null  object 
 8   createdbyuserorganization  1491773 non-null  object 
 9   createddate                1491773 non-null  object 
 10  requestsource              1491773 non-null  object 
 11  requesttype                1491773 non-null  object 
 12  assignto                   1467218 non-null  object 
 13  latitude    

<font size=5, color=green>Investigate on all the date columns<font>

<font size=3 color=gray>First of all we need to convert the type from object to date and then check to make sure there is not any date greater than today's date, then we can investigate for these criterias:<br>1. createddate <= servicedate<br>2. servicedate <= closeddate<br>3. closeddate <= updateddate<br>but we want to keep NaN value for these date columns<font>

<font size=2 color=Navy>Remove any record with date after today<font>

In [22]:
df311 = df311[(df311['createddate'] <= dt.now().strftime("%Y-%m-%d")) & (df311['createddate'].notnull())]

In [23]:
df311 = df311[(df311['servicedate'] <= dt.now().strftime("%Y-%m-%d")) & (df311['servicedate'].notnull())]

In [24]:
df311 = df311[(df311['closeddate'] <= dt.now().strftime("%Y-%m-%d")) & (df311['closeddate'].notnull())]

In [25]:
df311 = df311[(df311['updateddate'] <= dt.now().strftime("%Y-%m-%d")) & (df311['updateddate'].notnull())]

<font size=2 color=Navy>Convert the format from object to datetime<font>

In [26]:
df311['createddate'] = pd.to_datetime(df311['createddate'])
df311['servicedate'] = pd.to_datetime(df311['servicedate'])
df311['closeddate'] = pd.to_datetime(df311['closeddate'])
df311['updateddate'] = pd.to_datetime(df311['updateddate'])

<font size=2 color=Navy>Remove if there is any date violation<font>

In [27]:
df311 = df311[df311.createddate <= df311.servicedate]

In [28]:
df311 = df311[df311.servicedate <= df311.closeddate]

In [29]:
df311 = df311[df311.closeddate <= df311.updateddate]

In [30]:
df311.shape

(1150868, 34)

<font size=5, color=green>Create timedelta columns<font>

<font size=3 color=gray>Now after cleaning up all the datetime columns, it's time to create timedelta columns:<br>1. to_service: difference between createddate and servicedate<br>2. to_close: difference between createddate and closeddate<br>3. to_update: difference between createddate and updateddate<font>

In [31]:
df311['days_to_service'] = (df311.servicedate - df311.createddate).astype('timedelta64[D]')
df311['days_to_close'] = (df311.closeddate - df311.createddate).astype('timedelta64[D]')
df311['days_to_update'] = (df311.updateddate - df311.createddate).astype('timedelta64[D]')

<font size=5, color=green>Investigate on zipcodes<font>

<font size=3 color=gray>First we need to check if there is any recorde with null value or alphabetic value, then compare it with LA zipcodes (ZIPCode.csv is available and contains all the legitimate zipcodes for LA).<font>

<font size=2 color=Navy>Check for null and remove if there is any<font>

In [32]:
(df311.zipcode.isnull()).sum()

318

In [33]:
df311 = df311[df311.zipcode.notnull()]

<font size=2 color=Navy>Check for non-numeric records and remove if there is any<font>

In [34]:
wrong_zipcode = []
for z in df311.zipcode.unique().astype('str'):
    if re.search(r'[A-Za-z]', z):
        wrong_zipcode.append(z)

In [35]:
wrong_zipcode

['VE']

In [36]:
for code in wrong_zipcode:
    df311 = df311[df311['zipcode'] != code]

<font size=2 color=Navy>Compare with legitimate zipcodes and remove non-legitimates<font>

In [37]:
df311.zipcode = df311.zipcode.astype('int64')

In [39]:
df_laZipcode = pd.read_csv('ZIPCode.csv')

In [40]:
set1 = set(df_laZipcode.ZIPCODE.unique())
set2 = set(df311.zipcode.unique())
diff = set2 - set1

In [41]:
for codes in diff:
    df311 = df311[df311.zipcode != codes]

<font size=5, color=green>Investigate on categorical fields<font>

<font size=3 color=gray>Select useful columns, explanation of each field is available here:<br>https://data.lacity.org/City-Infrastructure-Service-Requests/MyLA311-Service-Request-Data-2020/rq3b-xjk8<br> then investigate on each feature and fix if there is any issue<font>

In [42]:
col = ['actiontaken', 'owner', 'requesttype', 'status', 'requestsource', 'createdbyuserorganization', 'mobileos', \
      'anonymous', 'assignto', 'addressverified', 'approximateaddress', 'apc', 'cd', 'cdmember', 'nc', 'ncname', \
      'policeprecinct']

In [43]:
print('***actiontaken***')
print('-----------------')
print('Type is:')
print(df311.actiontaken.dtypes)
print('Null reacords:')
print(df311.actiontaken.isnull().sum())
print('Unique values:')
print(df311.actiontaken.unique())

***actiontaken***
-----------------
Type is:
object
Null reacords:
0
Unique values:
['SR Created' 'Transferred' 'SR Updated' 'Information Provided'
 'Referred to Other Agency' 'Escalate to Supervisor' 'Consultation/3-way'
 'Unable to Assist' 'Status Provided']


In [44]:
print('***owner***')
print('-----------')
print('Type is:')
print(df311.owner.dtypes)
print('Null reacords:')
print(df311.owner.isnull().sum())
print('Unique values:')
print(df311.owner.unique())

***owner***
-----------
Type is:
object
Null reacords:
0
Unique values:
['LASAN' 'OCB' 'BSL' 'BOS']


In [45]:
print('***requesttype***')
print('-----------------')
print('Type is:')
print(df311.requesttype.dtypes)
print('Null reacords:')
print(df311.requesttype.isnull().sum())
print('Unique values:')
print(df311.requesttype.unique())

***requesttype***
-----------------
Type is:
object
Null reacords:
0
Unique values:
['Bulky Items' 'Graffiti Removal' 'Homeless Encampment'
 'Illegal Dumping Pickup' 'Electronic Waste' 'Multiple Streetlight Issue'
 'Single Streetlight Issue' 'Metal/Household Appliances'
 'Dead Animal Removal']


In [46]:
print('***status***')
print('------------')
print('Type is:')
print(df311.status.dtypes)
print('Null reacords:')
print(df311.status.isnull().sum())
print('Unique values:')
print(df311.status.unique())

***status***
------------
Type is:
object
Null reacords:
0
Unique values:
['Closed' 'Cancelled' 'Referred Out']


In [47]:
print('***requestsource***')
print('-------------------')
print('Type is:')
print(df311.requestsource.dtypes)
print('Null reacords:')
print(df311.requestsource.isnull().sum())
print('Unique values:')
print(df311.requestsource.unique())

***requestsource***
-------------------
Type is:
object
Null reacords:
0
Unique values:
['Mobile App' 'Self Service' 'Call' 'Email' 'Voicemail'
 'Driver Self Report' "Council's Office" 'Social'
 'Queue Initiated Customer Call' 'Walk-in' 'Twitter' 'Letter'
 'City Attorney' 'Web Form' 'TTY/ NexTalk' "Mayor's Office" 'Fax']


In [48]:
print('***createdbyuserorganization***')
print('-------------------------------')
print('Type is:')
print(df311.createdbyuserorganization.dtypes)
print('Null reacords:')
print(df311.createdbyuserorganization.isnull().sum())
print('Unique values:')
print(df311.createdbyuserorganization.unique())

***createdbyuserorganization***
-------------------------------
Type is:
object
Null reacords:
0
Unique values:
['Self Service' 'LASAN' 'Self Service_SAN' 'ITA' "Council's Office" 'OCB'
 'Proactive Insert' 'BSL' 'BSS' 'LASAN Franchise' 'LAAS' 'BOS'
 'BOS Franchise']


In [49]:
print('***mobileos***')
print('--------------')
print('Type is:')
print(df311.mobileos.dtypes)
print('Null reacords:')
print(df311.mobileos.isnull().sum())
print('Unique values:')
print(df311.mobileos.unique())

***mobileos***
--------------
Type is:
object
Null reacords:
807607
Unique values:
['Android' nan 'iOS']


In [50]:
print('***anonymous***')
print('---------------')
print('Type is:')
print(df311.anonymous.dtypes)
print('Null reacords:')
print(df311.anonymous.isnull().sum())
print('Unique values:')
print(df311.anonymous.unique())

***anonymous***
---------------
Type is:
object
Null reacords:
0
Unique values:
['N' 'Y']


In [51]:
print('***assignto***')
print('--------------')
print('Type is:')
print(df311.assignto.dtypes)
print('Null reacords:')
print(df311.assignto.isnull().sum())
print('Unique values:')
print(df311.assignto.unique())

***assignto***
--------------
Type is:
object
Null reacords:
3
Unique values:
['SLA' 'NDFY' 'EV' 'WV' 'WLA' 'NC' 'HB' 'NORTH' 'EAST' 'WEST' 'WVA' 'NEGB'
 'KYCC' 'NEGB_ELA' 'GAP' 'HBT' 'SOUTH' 'CCAC' 'CENTRAL' 'LACC' 'GAPBH'
 'PGS' 'CRCD' 'SGB' 'GCS' 'OCB' 'F-NE' 'F-DT' 'F-WV' 'F-NC' 'F-SEV' 'F-SE'
 'F-WLA' 'F-NEV' 'F-HB' nan 'WPD Enforcement' 'F-SLA']


<font size=2 color=Navy>Remove null records in this field<font>

In [52]:
df311 = df311[df311.assignto.notnull()]

In [53]:
print('***addressverified***')
print('---------------------')
print('Type is:')
print(df311.addressverified.dtypes)
print('Null reacords:')
print(df311.addressverified.isnull().sum())
print('Unique values:')
print(df311.addressverified.unique())

***addressverified***
---------------------
Type is:
object
Null reacords:
0
Unique values:
['Y']


In [54]:
print('***approximateaddress***')
print('------------------------')
print('Type is:')
print(df311.approximateaddress.dtypes)
print('Null reacords:')
print(df311.approximateaddress.isnull().sum())
print('Unique values:')
print(df311.approximateaddress.unique())

***approximateaddress***
------------------------
Type is:
object
Null reacords:
267043
Unique values:
[nan 'N' 'Y']


<font size=2 color=Navy>There won't be any report / dashboard based on this field, so we can keep it will all null values<font>

In [55]:
print('***apc***')
print('---------')
print('Type is:')
print(df311.apc.dtypes)
print('Null reacords:')
print(df311.apc.isnull().sum())
print('Unique values:')
print(df311.apc.unique())

***apc***
---------
Type is:
object
Null reacords:
9
Unique values:
['South Los Angeles APC' 'South Valley APC' 'West Los Angeles APC'
 'East Los Angeles APC' 'Harbor APC' 'North Valley APC' 'Central APC' nan]


<font size=2 color=Navy>Remove null records in this field<font>

In [56]:
df311 = df311[df311.apc.notnull()]

In [57]:
print('***cd***')
print('--------')
print('Type is:')
print(df311.cd.dtypes)
print('Null reacords:')
print(df311.cd.isnull().sum())
print('Unique values:')
print(df311.cd.unique())

***cd***
--------
Type is:
float64
Null reacords:
2
Unique values:
[ 8.  2.  3. 11. 14.  9. 15. 12.  6.  7. 10.  5.  1. 13.  4. nan]


<font size=2 color=Navy>Remove null records in this field and then change the type to int64<font>

In [58]:
df311 = df311[df311.cd.notnull()]
df311.cd = df311.cd.astype('int64')

In [59]:
print('***cdmember***')
print('--------------')
print('Type is:')
print(df311.cdmember.dtypes)
print('Null reacords:')
print(df311.cdmember.isnull().sum())
print('Unique values:')
print(df311.cdmember.unique())

***cdmember***
--------------
Type is:
object
Null reacords:
4
Unique values:
['Marqueece Harris-Dawson' 'Paul Krekorian' 'Bob Blumenfield' 'Mike Bonin'
 'Kevin de León' 'Curren D. Price Jr.' 'Joe Buscaino' 'John Lee'
 'Nury Martinez' 'Monica Rodriguez' 'Mark Ridley-Thomas' 'Paul Koretz'
 'Gilbert Cedillo' "Mitch O'Farrell" 'Nithya Raman' 'Jose Huizar'
 'Herb J. Wesson Jr.' 'David Ryu' nan 'Greig Smith' 'Mitchell Englander']


<font size=2 color=Navy>Remove null records in this field<font>

In [60]:
df311 = df311[df311.cdmember.notnull()]

In [61]:
print('***nc***')
print('--------')
print('Type is:')
print(df311.nc.dtypes)
print('Null reacords:')
print(df311.nc.isnull().sum())
print('Unique values:')
print(df311.nc.unique())

***nc***
--------
Type is:
float64
Null reacords:
4732
Unique values:
[ 87.  24.  23.  17.  67.  40.  86.  48.  nan  94. 113. 100.  96.  11.
   6.   9.  79. 119.  19.  41.  15.  58. 115. 112.  27.  61.  44.  95.
   4. 104. 110.   7.  64. 114.  26.  20. 101.   5.  76.  16.  29.  33.
  90.  50.  13.  47.  71.  28.  60.  21.  25.  42.  36.  34.  74. 128.
  30.  37.  84.  73.  75. 120.  52.  39.  55.  68.  14.  78. 125.  88.
  10.  66.  99. 109.   8.  18.  92. 124.  22.  53. 118.  77.  38.  80.
  81. 121.  97.  70.  63.  32.  93.  62. 111.  54. 127. 102.  91.  43.
 126.  46. 123.  65. 117.   2. 122.]


In [62]:
print('***ncname***')
print('------------')
print('Type is:')
print(df311.ncname.dtypes)
print('Null reacords:')
print(df311.ncname.isnull().sum())
print('Unique values:')
print(df311.ncname.unique())

***ncname***
------------
Type is:
object
Null reacords:
4732
Unique values:
['Empowerment Congress Southeast' 'NoHo' 'North Hollywood Northeast'
 'Tarzana' 'Mar Vista' 'Eagle Rock' 'CANNDU' 'LA32' nan
 'Northwest San Pedro' 'Northridge West' 'Panorama City'
 'Coastal San Pedro' 'West Hills' 'Arleta' 'Foothill Trails District'
 'Empowerment Congress West' 'Greater Wilshire' 'Lake Balboa'
 'Historic Highland Park' 'Reseda' 'Mid City West' 'Palms'
 'North Hills East' 'Studio City' 'South Robertson' 'Echo Park'
 'Central San Pedro' 'Granada Hills North' 'Olympic Park'
 'Central Alameda' 'Pacoima' 'Bel Air-Beverly Crest' 'Porter Ranch'
 'Sherman Oaks' 'Van Nuys' 'Mission Hills' 'Sylmar' 'Pico Union'
 'Woodland Hills-Warner Center' 'Hollywood Hills West'
 'Hollywood Studio District' 'Harbor Gateway North' 'Boyle Heights'
 'Canoga Park' 'Lincoln Heights' 'Westchester/Playa' 'Greater Toluca Lake'
 'P.I.C.O.' 'Greater Valley Glen' 'Valley Village' 'Arroyo Seco'
 'Los Feliz' 'East Hollywood' 'U

In [63]:
df311[['nc', 'ncname']][df311.nc.isnull() & df311.ncname.isnull()].shape

(4732, 2)

<font size=2 color=Navy>There are many records which has both nc and ncmember with null value, remove all of them<font>

In [64]:
df311 = df311[df311.nc.notnull() & df311.ncname.notnull()].reset_index()

<font size=2 color=Navy>Now cleaning up the ncname column<font>

In [65]:
print('***ncname***')
print('------------')
print('Type is:')
print(df311.ncname.dtypes)
print('Null reacords:')
print(df311.ncname.isnull().sum())
print('Count unique values:')
print(df311.ncname.nunique())
print('Unique values:')
print(df311.ncname.unique())

***ncname***
------------
Type is:
object
Null reacords:
0
Count unique values:
203
Unique values:
['Empowerment Congress Southeast' 'NoHo' 'North Hollywood Northeast'
 'Tarzana' 'Mar Vista' 'Eagle Rock' 'CANNDU' 'LA32' 'Northwest San Pedro'
 'Northridge West' 'Panorama City' 'Coastal San Pedro' 'West Hills'
 'Arleta' 'Foothill Trails District' 'Empowerment Congress West'
 'Greater Wilshire' 'Lake Balboa' 'Historic Highland Park' 'Reseda'
 'Mid City West' 'Palms' 'North Hills East' 'Studio City'
 'South Robertson' 'Echo Park' 'Central San Pedro' 'Granada Hills North'
 'Olympic Park' 'Central Alameda' 'Pacoima' 'Bel Air-Beverly Crest'
 'Porter Ranch' 'Sherman Oaks' 'Van Nuys' 'Mission Hills' 'Sylmar'
 'Pico Union' 'Woodland Hills-Warner Center' 'Hollywood Hills West'
 'Hollywood Studio District' 'Harbor Gateway North' 'Boyle Heights'
 'Canoga Park' 'Lincoln Heights' 'Westchester/Playa' 'Greater Toluca Lake'
 'P.I.C.O.' 'Greater Valley Glen' 'Valley Village' 'Arroyo Seco'
 'Los Feliz' 'E

In [66]:
print('***policeprecinct***')
print('--------------------')
print('Type is:')
print(df311.policeprecinct.dtypes)
print('Null reacords:')
print(df311.policeprecinct.isnull().sum())
print('Unique values:')
print(df311.policeprecinct.unique())

***policeprecinct***
--------------------
Type is:
object
Null reacords:
0
Unique values:
['SOUTHEAST' 'NORTH HOLLYWOOD' 'WEST VALLEY' 'PACIFIC' 'NORTHEAST'
 '77TH STREET' 'HOLLENBECK' 'HARBOR' 'DEVONSHIRE' 'MISSION' 'TOPANGA'
 'FOOTHILL' 'SOUTHWEST' 'WILSHIRE' 'WEST LOS ANGELES' 'RAMPART' 'OLYMPIC'
 'NEWTON' 'VAN NUYS' 'HOLLYWOOD' 'CENTRAL']


<font size=5, color=green>Save the clean data set to a csv file<font>

In [ ]:
df311.to_csv('clean_311data.csv', index = False)